# Udacity Data Engineering Capstone: Honeypot Data Warehouse

By: Tyler Byers  
Date: 2019-11-18  
tybyers at gmail dot com

## About

This project writeup is submitted to fulfill requirements for the Udacity Data Engineering Capstone project. In this project, I used data sets from three different sources, totaling about 1.25 million rows of data, to create a Data Warehouse on Amazon Redshift. This data pulls together data from a data set compiling log data from a few different [honeypots] (JSON), [AlienVault](https://www.alienvault.com/) reputation data (#-delimited), and IP geolocation data from [freegeoip.app](https://freegeoip.app/) (API/CSV). 

If we were to imagine this Data Warehouse had a business implication, we could easily imagine a company 

TODO^^

## Data Sets

## Data Model

<img src="./data/honeypot_db_schema.png">

## Process to Build Data Warehouse

## Hypotheticals for Scaling

## Demo: How to Setup, Run

## Examples, Data Table Stats